# 0 Load data


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from function import *
from function.global_varibles import *
import glob


%load_ext autoreload
%autoreload 2


df_pv_npv = pd.read_csv(PATHS['df_pv_npv'])
df_agricultural_npv = pd.read_csv(PATHS['df_agricultural_npv'])
df_afforestation_npv = pd.read_csv(PATHS['df_afforestation_npv'])
df_natural_npv = pd.read_csv(PATHS['df_natural_npv'])

# 加载策略数据
df_weight = pd.read_csv(PATHS['data_weighted_density'])
df_strategies=pd.read_csv(PATHS['data_strategies'])
df_net_benefit=pd.read_csv(PATHS['data_prediction_carbon_benefit'])

# 加载经济预期数据
df_economic=pd.read_csv(PATHS['df_economic_feasibility'])





gogogo


# ## 1. 撂荒地用于光伏建设再开发的决策优先级分区

**预期**：通过环境潜力、减排潜力、经济潜力（三类可行性）提供全面的决策信息。

**体现光伏带来的效益**：让不同区域或国家决策者能够更清晰知道自身在主要维度上哪一块是短板，能够大概了解：在本地再开发xx%比例的撂荒地用于光伏建设，是边际效益变化最为"剧烈"的区域
- 单维度（三个图）：x轴是分位数变化（这里的测度可以是由跟着主图的三变量相关性）；y轴累积效应（如果区域效益差距过大、可以考虑用百分比表示）；图中加上斜率，折线代表各个区域（决策主体）
- 多维度（一个图）：三个维度的轴汇总在一起，观测联合边际效应变化区域

**体现光伏收益的时效**：让决策者知道撂荒地开发在2020-2050期间大概要花多久才能起到较好的政策效果
- 在2020-2050年30年期间，随着时间变化，各个决策主体在开发10%、20%..一直到100%等不同优先区的情况下，随着时间变化各维度效益发展路径。（可以在图中以凸包、区间的形式表达）



以下是三类潜力的变量申明：

df_economic[Policy_category]: 'P2a', 'P1b', 'P1a', 'P4', 'P2c', 'P1c', 'P2', 'P1d', 'P3b', 'P3c', 'P3a'

经济潜力需要的变量：df_economic[net_npv_usd] （每个pixel可能有265个值，需要根据根据Policy_category group并计算各个policy以及总体的mean均值）
环境潜力需要的变量: df_weight[env_scores]
减排潜力所需的变量：df_weight[Expectation_net_benefit]

每个中都有analysis_year标注年份2020,2030,2040,2050

In [2]:
df_economic['analysis_year'].unique()

array([2020, 2030, 2040, 2050], dtype=int64)

In [20]:
df_natural_npv

,pixel_id,lat,lon,analysis_year,rcp_category,cumulative_npv_usd,cumulative_cost_usd
0,"(25.004168, -107.54583)",25.004168,-107.545830,2020,RCP2.6,-41.0,41.0
1,"(25.0125, -107.6125)",25.012500,-107.612500,2020,RCP2.6,-41.0,41.0
2,"(25.0125, -107.4875)",25.012500,-107.487500,2020,RCP2.6,-41.0,41.0
3,"(25.029167, -107.6125)",25.029167,-107.612500,2020,RCP2.6,-41.0,41.0
4,"(25.029167, -105.4125)",25.029167,-105.412500,2020,RCP2.6,-146.0,146.0
...,...,...,...,...,...,...,...
1124987,"(48.1875, -65.8375)",48.187500,-65.837500,2050,RCP8.5,-165.0,165.0
1124988,"(48.579166, -67.9875)",48.579166,-67.987500,2050,RCP8.5,-165.0,165.0
1124989,"(48.745834, -72.4375)",48.745834,-72.437500,2050,RCP8.5,-165.0,165.0
1124990,"(48.7625, -72.4375)",48.762500,-72.437500,2050,RCP8.5,-165.0,165.0


# 情景检查与匹配

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from function import *
import glob




df_r10 = pd.read_csv(r'D:\xarray\aligned2\economic_cost\national_growth_rate\AR6_Scenarios_Database_R10_regions_v1.1.csv')
df_world = pd.read_csv(r'D:\xarray\aligned2\economic_cost\national_growth_rate\AR6_Scenarios_Database_World_v1.1.csv')
df_indicators = pd.read_csv(r'D:\xarray\aligned2\economic_cost\national_growth_rate\AR6_Scenarios_Database_metadata_indicators_v1.1.csv')
df_data = pd.read_csv(r'data\4.data_prediction_net_benefit.csv')









gogogo


In [ ]:
# Create scenario identifiers and map to categories
# Get unique regions
# accross all regions 285 
unique_regions = df_r10['Region'].unique()

# Get scenarios from df_indicators
indicator_scenarios = set(df_indicators['Model'].astype(str) + '_' + df_indicators['Scenario'].astype(str))

# Find scenarios common across all regions
scenarios_by_region = {}
for region in unique_regions:
    df_region = df_r10[df_r10['Region'] == region]
    scenarios_region = set(df_region['Model'].astype(str) + '_' + df_region['Scenario'].astype(str))
    scenarios_by_region[region] = scenarios_region

# Get intersection of scenarios across all regions and indicators
common_scenarios = set.intersection(*scenarios_by_region.values(), indicator_scenarios)

# Create mapping of scenarios to categories
scenario_categories = {}
for _, row in df_indicators.iterrows():
    scenario_id = str(row['Model']) + '_' + str(row['Scenario'])
    if scenario_id in common_scenarios:
        scenario_categories[scenario_id] = row['Category']

# Define target variables to extract from r10 dataset
target_variables = {
    'capital_cost': 'Capital Cost|Electricity|Solar|PV',
    'om_fixed_cost': 'OM Cost|Fixed|Electricity|Solar|PV', 
    'electricity_price': 'Price|Secondary Energy|Electricity'
}

target_years = [2010, 2020, 2030, 2040, 2050]

# Process each region separately
region_scenarios = {}
for region in unique_regions:
    print(f"\nRegion: {region}")
    
    # Filter for current region
    df_r10_region = df_r10[df_r10['Region'] == region]
    
    # Filter for target variables and common scenarios
    mask_variables = df_r10_region['Variable'].isin(target_variables.values())
    mask_scenarios = df_r10_region['Model'].astype(str) + '_' + df_r10_region['Scenario'].astype(str)
    mask_scenarios = mask_scenarios.isin(common_scenarios)
    df_r10_filtered = df_r10_region[mask_variables & mask_scenarios]
    
    # Analyze variable and year availability for each scenario
    scenario_stats = {}
    for model_scenario in df_r10_filtered.groupby(['Model', 'Scenario']):
        model, scenario = model_scenario[0]
        scenario_id = f"{model}_{scenario}"
        scenario_data = model_scenario[1]
        
        # Skip if scenario not in categories mapping
        if scenario_id not in scenario_categories:
            continue
            
        var_availability = {}
        year_availability = {year: 0 for year in target_years}
        
        for var_name, var in target_variables.items():
            var_data = scenario_data[scenario_data['Variable'] == var]
            valid_years = 0
            
            for year in target_years:
                year_str = str(year)
                if year_str in var_data.columns and len(var_data) > 0:
                    value = var_data[year_str].iloc[0]
                    if not pd.isna(value):
                        valid_years += 1
                        year_availability[year] += 1
                        
            var_availability[var_name] = valid_years
            
        scenario_stats[scenario_id] = {
            'category': scenario_categories[scenario_id],
            'model': model,
            'scenario': scenario,
            'variables': var_availability,
            'years': year_availability,
            'total_vars': sum(1 for v in var_availability.values() if v >= 2)
        }
    
    # Print summary statistics
    print("\nVariable availability patterns:")
    pattern_counts = {}
    for scenario_id, stats in scenario_stats.items():
        pattern = tuple(v >= 2 for v in stats['variables'].values())
        if pattern not in pattern_counts:
            pattern_counts[pattern] = {'count': 0, 'categories': {}, 'years': {y:0 for y in target_years}}
        pattern_counts[pattern]['count'] += 1
        cat = stats['category']
        pattern_counts[pattern]['categories'][cat] = pattern_counts[pattern]['categories'].get(cat, 0) + 1
        # Add year counts
        for year, count in stats['years'].items():
            pattern_counts[pattern]['years'][year] += count
    
    for pattern, counts in pattern_counts.items():
        var_pattern = [var_name for var_name, has_data in zip(target_variables.keys(), pattern) if has_data]
        print(f"\nPattern: {var_pattern}")
        print(f"Total scenarios: {counts['count']}")
        print("By category:")
        for cat, cat_count in counts['categories'].items():
            print(f"  {cat}: {cat_count}")
        print("By year:")
        for year, year_count in counts['years'].items():
            print(f"  {year}: {year_count}")
            
    # Store complete scenarios (2 or more valid variables)
    complete_scenarios_region = [(stats['model'], stats['scenario']) 
                               for stats in scenario_stats.values() 
                               if stats['total_vars'] >= 2]
    region_scenarios[region] = complete_scenarios_region


Region: R10AFRICA

Variable availability patterns:

Pattern: ['capital_cost']
Total scenarios: 1
By category:
  C7: 1
By year:
  2010: 0
  2020: 1
  2030: 1
  2040: 1
  2050: 1

Pattern: ['capital_cost', 'om_fixed_cost', 'electricity_price']
Total scenarios: 208
By category:
  C1: 29
  C3: 63
  C7: 27
  C6: 14
  C8: 6
  C2: 34
  C5: 27
  C4: 8
By year:
  2010: 624
  2020: 624
  2030: 624
  2040: 624
  2050: 624

Pattern: ['capital_cost', 'electricity_price']
Total scenarios: 42
By category:
  C3: 21
  C7: 8
  C1: 4
  C5: 7
  C4: 2
By year:
  2010: 84
  2020: 84
  2030: 84
  2040: 84
  2050: 84

Pattern: ['electricity_price']
Total scenarios: 15
By category:
  C4: 4
  C5: 4
  C6: 4
  C3: 3
By year:
  2010: 15
  2020: 15
  2030: 15
  2040: 15
  2050: 15

Region: R10CHINA+

Variable availability patterns:

Pattern: ['capital_cost']
Total scenarios: 1
By category:
  C7: 1
By year:
  2010: 0
  2020: 1
  2030: 1
  2040: 1
  2050: 1

Pattern: ['capital_cost', 'om_fixed_cost', 'electricity_pr